In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    # rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size,
               states, actions, next_states, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ ~s', Q = ~env(s, a)
    ################################################ ~s', Q = ~env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # min
    #################################################### s'', Q' = ~env(s', ~a')
    for _ in range(11):
        next_states = tf.nn.tanh(next_states_logits)
        next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
        next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                       action_size=action_size, state_size=state_size, reuse=True)
        aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
        eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                        labels=tf.zeros_like(rates))) # min
        aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                         labels=tf.ones_like(rates))) # min
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.dones, self.rates = model_input(state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [10]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [11]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [12]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
# gamma = 0.99                   # future reward discount

In [13]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, 
              e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [14]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                #rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                #rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               #model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               #model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              #model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print('dones:', 
            #       len(dones), np.count_nonzero(dones), 
            #       len(dones1), np.count_nonzero(dones1), 
            #       len(dones2), np.count_nonzero(dones2))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:17.0000 R:17.0000 rate:0.0340 aloss:0.7241 eloss:10.5700 aloss2:9.5478 exploreP:0.9983
Episode:1 meanR:18.0000 R:19.0000 rate:0.0380 aloss:0.7181 eloss:10.5675 aloss2:9.5508 exploreP:0.9964
Episode:2 meanR:16.6667 R:14.0000 rate:0.0280 aloss:0.7256 eloss:10.6239 aloss2:9.5057 exploreP:0.9951
Episode:3 meanR:15.5000 R:12.0000 rate:0.0240 aloss:0.7237 eloss:10.5025 aloss2:9.5874 exploreP:0.9939
Episode:4 meanR:17.0000 R:23.0000 rate:0.0460 aloss:0.7122 eloss:10.6269 aloss2:9.5057 exploreP:0.9916
Episode:5 meanR:19.5000 R:32.0000 rate:0.0640 aloss:0.7168 eloss:10.5480 aloss2:9.5530 exploreP:0.9885
Episode:6 meanR:21.7143 R:35.0000 rate:0.0700 aloss:0.7175 eloss:10.6006 aloss2:9.5034 exploreP:0.9851
Episode:7 meanR:21.1250 R:17.0000 rate:0.0340 aloss:0.7185 eloss:10.6728 aloss2:9.4477 exploreP:0.9834
Episode:8 meanR:19.8889 R:10.0000 rate:0.0200 aloss:0.7132 eloss:10.6381 aloss2:9.4751 exploreP:0.9824
Episode:9 meanR:19.2000 R:13.0000 rate:0.0260 aloss:0.7114 eloss:10.5744 

Episode:79 meanR:20.4625 R:26.0000 rate:0.0520 aloss:0.7035 eloss:5.0355 aloss2:18.9700 exploreP:0.8505
Episode:80 meanR:20.4568 R:20.0000 rate:0.0400 aloss:0.7069 eloss:4.9788 aloss2:19.1777 exploreP:0.8488
Episode:81 meanR:20.6098 R:33.0000 rate:0.0660 aloss:0.6963 eloss:4.8814 aloss2:19.4060 exploreP:0.8461
Episode:82 meanR:20.5060 R:12.0000 rate:0.0240 aloss:0.7042 eloss:4.8595 aloss2:19.5165 exploreP:0.8451
Episode:83 meanR:20.4524 R:16.0000 rate:0.0320 aloss:0.7320 eloss:4.8213 aloss2:19.6205 exploreP:0.8437
Episode:84 meanR:20.4941 R:24.0000 rate:0.0480 aloss:0.7411 eloss:4.7464 aloss2:19.8399 exploreP:0.8417
Episode:85 meanR:20.4535 R:17.0000 rate:0.0340 aloss:0.7145 eloss:4.6177 aloss2:20.2296 exploreP:0.8403
Episode:86 meanR:20.4828 R:23.0000 rate:0.0460 aloss:0.7150 eloss:4.5702 aloss2:20.4419 exploreP:0.8384
Episode:87 meanR:20.3864 R:12.0000 rate:0.0240 aloss:0.7169 eloss:4.5126 aloss2:20.6074 exploreP:0.8374
Episode:88 meanR:20.3708 R:19.0000 rate:0.0380 aloss:0.7046 elos

Episode:158 meanR:31.2700 R:16.0000 rate:0.0320 aloss:0.9088 eloss:1.6281 aloss2:54.7913 exploreP:0.6525
Episode:159 meanR:31.6100 R:45.0000 rate:0.0900 aloss:0.9360 eloss:1.6306 aloss2:55.4261 exploreP:0.6496
Episode:160 meanR:31.7200 R:30.0000 rate:0.0600 aloss:0.8897 eloss:1.6422 aloss2:56.0479 exploreP:0.6477
Episode:161 meanR:32.3600 R:82.0000 rate:0.1640 aloss:0.8727 eloss:1.6138 aloss2:56.8331 exploreP:0.6425
Episode:162 meanR:32.5500 R:37.0000 rate:0.0740 aloss:0.8780 eloss:1.6078 aloss2:57.2102 exploreP:0.6401
Episode:163 meanR:32.5000 R:12.0000 rate:0.0240 aloss:1.0458 eloss:1.5923 aloss2:57.7537 exploreP:0.6394
Episode:164 meanR:32.9100 R:56.0000 rate:0.1120 aloss:0.8533 eloss:1.6444 aloss2:57.9917 exploreP:0.6358
Episode:165 meanR:33.6000 R:84.0000 rate:0.1680 aloss:0.9249 eloss:1.5898 aloss2:59.3521 exploreP:0.6306
Episode:166 meanR:33.7000 R:19.0000 rate:0.0380 aloss:0.9331 eloss:1.5595 aloss2:60.0965 exploreP:0.6294
Episode:167 meanR:34.0600 R:55.0000 rate:0.1100 aloss:1

Episode:236 meanR:55.9100 R:73.0000 rate:0.1460 aloss:0.7532 eloss:1.5319 aloss2:79.7482 exploreP:0.4141
Episode:237 meanR:55.8600 R:25.0000 rate:0.0500 aloss:0.7458 eloss:1.5278 aloss2:77.2070 exploreP:0.4131
Episode:238 meanR:55.6300 R:44.0000 rate:0.0880 aloss:0.7323 eloss:1.5498 aloss2:74.9712 exploreP:0.4113
Episode:239 meanR:55.8700 R:40.0000 rate:0.0800 aloss:0.7381 eloss:1.5312 aloss2:73.6375 exploreP:0.4097
Episode:240 meanR:56.0000 R:27.0000 rate:0.0540 aloss:0.6959 eloss:1.5759 aloss2:72.7116 exploreP:0.4087
Episode:241 meanR:56.4700 R:60.0000 rate:0.1200 aloss:0.7124 eloss:1.5867 aloss2:72.1656 exploreP:0.4063
Episode:242 meanR:56.4500 R:53.0000 rate:0.1060 aloss:0.7302 eloss:1.5487 aloss2:72.6788 exploreP:0.4042
Episode:243 meanR:56.4600 R:33.0000 rate:0.0660 aloss:0.7016 eloss:1.4882 aloss2:74.6410 exploreP:0.4029
Episode:244 meanR:56.9000 R:110.0000 rate:0.2200 aloss:0.6973 eloss:1.5563 aloss2:75.6736 exploreP:0.3986
Episode:245 meanR:57.1800 R:68.0000 rate:0.1360 aloss:

Episode:314 meanR:34.1500 R:11.0000 rate:0.0220 aloss:0.7542 eloss:1.4797 aloss2:105.7838 exploreP:0.3426
Episode:315 meanR:33.5100 R:12.0000 rate:0.0240 aloss:0.7619 eloss:1.4332 aloss2:105.1449 exploreP:0.3422
Episode:316 meanR:32.8400 R:11.0000 rate:0.0220 aloss:0.7746 eloss:1.4301 aloss2:105.3509 exploreP:0.3418
Episode:317 meanR:32.0300 R:11.0000 rate:0.0220 aloss:0.7095 eloss:1.3995 aloss2:105.6118 exploreP:0.3415
Episode:318 meanR:31.6000 R:9.0000 rate:0.0180 aloss:0.7291 eloss:1.4218 aloss2:106.1879 exploreP:0.3412
Episode:319 meanR:30.8300 R:13.0000 rate:0.0260 aloss:0.7439 eloss:1.4371 aloss2:105.6040 exploreP:0.3407
Episode:320 meanR:30.5800 R:18.0000 rate:0.0360 aloss:0.7322 eloss:1.4987 aloss2:106.7300 exploreP:0.3401
Episode:321 meanR:30.3600 R:11.0000 rate:0.0220 aloss:0.7680 eloss:1.4325 aloss2:106.7129 exploreP:0.3398
Episode:322 meanR:29.9200 R:12.0000 rate:0.0240 aloss:0.7781 eloss:1.4510 aloss2:106.8702 exploreP:0.3394
Episode:323 meanR:29.3000 R:9.0000 rate:0.0180 

...

Episode:2856 meanR:297.5500 R:284.0000 rate:0.5680 aloss:0.0828 eloss:1081.2539 aloss2:34.1080 exploreP:0.0100
Episode:2857 meanR:293.0700 R:52.0000 rate:0.1040 aloss:0.0836 eloss:1069.1046 aloss2:34.3343 exploreP:0.0100
Episode:2858 meanR:288.5700 R:50.0000 rate:0.1000 aloss:0.0802 eloss:1062.1425 aloss2:34.0670 exploreP:0.0100
Episode:2859 meanR:293.3000 R:500.0000 rate:1.0000 aloss:0.0840 eloss:1067.9198 aloss2:33.5641 exploreP:0.0100
Episode:2860 meanR:288.7400 R:27.0000 rate:0.0540 aloss:0.0868 eloss:1070.9347 aloss2:33.6635 exploreP:0.0100
Episode:2861 meanR:283.9800 R:24.0000 rate:0.0480 aloss:0.0799 eloss:1070.5065 aloss2:33.6665 exploreP:0.0100
Episode:2862 meanR:280.2100 R:35.0000 rate:0.0700 aloss:0.0765 eloss:1077.4110 aloss2:33.1618 exploreP:0.0100
Episode:2863 meanR:284.9400 R:500.0000 rate:1.0000 aloss:0.0816 eloss:1120.2052 aloss2:28.5564 exploreP:0.0100
Episode:2864 meanR:285.0600 R:40.0000 rate:0.0800 aloss:0.0804 eloss:1142.1798 aloss2:24.4165 exploreP:0.0100
Episode

Episode:2931 meanR:271.2200 R:53.0000 rate:0.1060 aloss:0.0785 eloss:973.0825 aloss2:30.3887 exploreP:0.0100
Episode:2932 meanR:270.6600 R:444.0000 rate:0.8880 aloss:0.0796 eloss:1119.4364 aloss2:27.0363 exploreP:0.0100
Episode:2933 meanR:266.9300 R:38.0000 rate:0.0760 aloss:0.0828 eloss:1041.8219 aloss2:27.5435 exploreP:0.0100
Episode:2934 meanR:271.5400 R:500.0000 rate:1.0000 aloss:0.0805 eloss:1363.5668 aloss2:25.7481 exploreP:0.0100
Episode:2935 meanR:271.7100 R:55.0000 rate:0.1100 aloss:0.0788 eloss:1552.4934 aloss2:24.6329 exploreP:0.0100
Episode:2936 meanR:276.1800 R:500.0000 rate:1.0000 aloss:0.0784 eloss:1543.5145 aloss2:24.9843 exploreP:0.0100
Episode:2937 meanR:280.6500 R:500.0000 rate:1.0000 aloss:0.0786 eloss:1530.1283 aloss2:25.4284 exploreP:0.0100
Episode:2938 meanR:285.2400 R:500.0000 rate:1.0000 aloss:0.0788 eloss:1524.2972 aloss2:25.7647 exploreP:0.0100
Episode:2939 meanR:280.6600 R:42.0000 rate:0.0840 aloss:0.0849 eloss:1524.9973 aloss2:25.7966 exploreP:0.0100
Episod

Episode:3006 meanR:296.9900 R:81.0000 rate:0.1620 aloss:0.0729 eloss:1716.6310 aloss2:12.1330 exploreP:0.0100
Episode:3007 meanR:296.9900 R:500.0000 rate:1.0000 aloss:0.0766 eloss:1719.1320 aloss2:12.1587 exploreP:0.0100
Episode:3008 meanR:301.6400 R:500.0000 rate:1.0000 aloss:0.0742 eloss:1725.7476 aloss2:12.1111 exploreP:0.0100
Episode:3009 meanR:301.6400 R:500.0000 rate:1.0000 aloss:0.0748 eloss:1740.3766 aloss2:12.5517 exploreP:0.0100
Episode:3010 meanR:301.6400 R:500.0000 rate:1.0000 aloss:0.0737 eloss:1732.5093 aloss2:12.9189 exploreP:0.0100
Episode:3011 meanR:306.1900 R:500.0000 rate:1.0000 aloss:0.0741 eloss:1732.8395 aloss2:12.9544 exploreP:0.0100
Episode:3012 meanR:305.9900 R:480.0000 rate:0.9600 aloss:0.0754 eloss:1728.7633 aloss2:13.0497 exploreP:0.0100
Episode:3013 meanR:306.1300 R:53.0000 rate:0.1060 aloss:0.0699 eloss:1721.8684 aloss2:12.9791 exploreP:0.0100
Episode:3014 meanR:301.5400 R:41.0000 rate:0.0820 aloss:0.0715 eloss:1720.6851 aloss2:12.7143 exploreP:0.0100
Epis

Episode:3081 meanR:345.1200 R:48.0000 rate:0.0960 aloss:0.0738 eloss:164.8654 aloss2:19.9083 exploreP:0.0100
Episode:3082 meanR:340.5200 R:40.0000 rate:0.0800 aloss:0.0674 eloss:62.1908 aloss2:21.0756 exploreP:0.0100
Episode:3083 meanR:345.1100 R:500.0000 rate:1.0000 aloss:0.0649 eloss:157.6335 aloss2:27.3613 exploreP:0.0100
Episode:3084 meanR:345.1100 R:500.0000 rate:1.0000 aloss:0.0671 eloss:201.9774 aloss2:15.8446 exploreP:0.0100
Episode:3085 meanR:349.5600 R:500.0000 rate:1.0000 aloss:0.0661 eloss:60.7305 aloss2:43.8146 exploreP:0.0100
Episode:3086 meanR:349.5600 R:500.0000 rate:1.0000 aloss:0.0678 eloss:12.0939 aloss2:56.7111 exploreP:0.0100
Episode:3087 meanR:345.7400 R:48.0000 rate:0.0960 aloss:0.0709 eloss:14.0907 aloss2:60.0806 exploreP:0.0100
Episode:3088 meanR:342.2400 R:36.0000 rate:0.0720 aloss:0.0656 eloss:13.6993 aloss2:59.9519 exploreP:0.0100
Episode:3089 meanR:342.2400 R:500.0000 rate:1.0000 aloss:0.0682 eloss:13.1762 aloss2:60.0506 exploreP:0.0100
Episode:3090 meanR:3

Episode:3157 meanR:335.7000 R:500.0000 rate:1.0000 aloss:0.0697 eloss:9.0639 aloss2:54.7457 exploreP:0.0100
Episode:3158 meanR:340.1700 R:500.0000 rate:1.0000 aloss:0.0704 eloss:8.7239 aloss2:60.5892 exploreP:0.0100
Episode:3159 meanR:335.6100 R:44.0000 rate:0.0880 aloss:0.0678 eloss:8.0917 aloss2:61.6809 exploreP:0.0100
Episode:3160 meanR:340.1500 R:500.0000 rate:1.0000 aloss:0.0699 eloss:8.2547 aloss2:60.6401 exploreP:0.0100
Episode:3161 meanR:340.1500 R:500.0000 rate:1.0000 aloss:0.0679 eloss:8.5363 aloss2:60.7544 exploreP:0.0100
Episode:3162 meanR:337.9600 R:281.0000 rate:0.5620 aloss:0.0661 eloss:8.2814 aloss2:62.7744 exploreP:0.0100
Episode:3163 meanR:337.9600 R:500.0000 rate:1.0000 aloss:0.0668 eloss:8.3204 aloss2:61.2149 exploreP:0.0100
Episode:3164 meanR:337.9600 R:500.0000 rate:1.0000 aloss:0.0657 eloss:8.2707 aloss2:62.2118 exploreP:0.0100
Episode:3165 meanR:337.9600 R:500.0000 rate:1.0000 aloss:0.0670 eloss:8.1035 aloss2:62.3739 exploreP:0.0100
Episode:3166 meanR:339.3900 R

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [39]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
